In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected

from matplotlib import pyplot as plt
from matplotlib.patches import Circle

from sklearn.model_selection import train_test_split
from pylab import *

import pickle
import loadnotebook
from predictionhelper import *

from bayes_opt import BayesianOptimization
import sklearn.metrics as metric

import warnings
warnings.filterwarnings('ignore')

import random 
def shuffle_forward(l):
    order = np.arange(l); 
    random.shuffle(order)
    return order

def shuffle_backward(order):
    r_order = np.zeros_like(order)
    for i, j in enumerate(order):
        r_order[j] = i
    return r_order

importing Jupyter notebook from predictionhelper.ipynb


In [2]:
df = pd.DataFrame.from_csv('../data/ujiindoorloc/trainingData.csv').reset_index()
df_val = pd.DataFrame.from_csv('../data/ujiindoorloc/validationData.csv').reset_index()

In [3]:
df.BUILDINGID.unique(), df.FLOOR.unique()

(array([1, 0, 2]), array([2, 0, 3, 4, 1]))

In [4]:
df_WAP = pd.DataFrame()
for i in range(1,521) :
    AP = 'WAP%03d' % i
    df_WAP_001 = df[df[AP] < 100].reset_index(drop=True)
    df_WAP_001 = df_WAP_001[[AP, 'LONGITUDE', 'LATITUDE', 'BUILDINGID', 'FLOOR']]
    df_WAP_001 = df_WAP_001.rename(columns={AP : 'WAP'})
    df_WAP = df_WAP.append(df_WAP_001, ignore_index=True)
    
df_WAP = df_WAP.groupby(['LONGITUDE', 'LATITUDE', 'BUILDINGID', 'FLOOR']).mean().reset_index()
df_WAP.WAP = df_WAP.WAP + 100
df_WAP

,LONGITUDE,LATITUDE,BUILDINGID,FLOOR,WAP
0,-7691.338400,4.864928e+06,0,1,17.034483
1,-7691.338400,4.864928e+06,0,2,20.265306
2,-7691.338400,4.864928e+06,0,3,21.292191
3,-7689.513400,4.864929e+06,0,1,17.066148
4,-7689.513400,4.864929e+06,0,2,21.635088
5,-7689.513400,4.864929e+06,0,3,19.508961
6,-7685.723500,4.864930e+06,0,0,20.764423
7,-7684.298100,4.864932e+06,0,1,24.302251
8,-7684.298100,4.864932e+06,0,2,26.261860
9,-7684.298100,4.864932e+06,0,3,23.521576


In [ ]:
df_WAP_pair = pd.DataFrame()
df_WAP_pair['X1_LONG'] = df_WAP.LONGITUDE.repeat(len(df_WAP))
df_WAP_pair['X1_LAT'] = df_WAP.LATITUDE.repeat(len(df_WAP))
df_WAP_pair['X1_B'] = df_WAP.BUILDINGID.repeat(len(df_WAP))
df_WAP_pair['X1_F'] = df_WAP.FLOOR.repeat(len(df_WAP))
df_WAP_pair['X1_WAP'] = df_WAP.WAP.repeat(len(df_WAP))
df_WAP_pair['X2_LONG'] = pd.np.tile(df_WAP.LONGITUDE, len(df_WAP)) 
df_WAP_pair['X2_LAT'] = pd.np.tile(df_WAP.LATITUDE, len(df_WAP)) 
df_WAP_pair['X2_B'] = pd.np.tile(df_WAP.BUILDINGID, len(df_WAP)) 
df_WAP_pair['X2_F'] = pd.np.tile(df_WAP.FLOOR, len(df_WAP)) 
df_WAP_pair['X2_WAP'] = pd.np.tile(df_WAP.WAP, len(df_WAP)) 
df_WAP_pair['Y'] = abs(df_WAP_pair.X1_WAP - df_WAP_pair.X2_WAP)
df_WAP_pair = df_WAP_pair.drop(['X1_WAP', 'X2_WAP'], axis=1)
df_WAP_pair = df_WAP_pair[df_WAP_pair.Y != 0].reset_index(drop=True)
df_WAP_pair

,X1_LONG,X1_LAT,X1_B,X1_F,X2_LONG,X2_LAT,X2_B,X2_F,Y
0,-7691.33840,4.864928e+06,0,1,-7691.338400,4.864928e+06,0,2,3.230823
1,-7691.33840,4.864928e+06,0,1,-7691.338400,4.864928e+06,0,3,4.257709
2,-7691.33840,4.864928e+06,0,1,-7689.513400,4.864929e+06,0,1,0.031665
3,-7691.33840,4.864928e+06,0,1,-7689.513400,4.864929e+06,0,2,4.600605
4,-7691.33840,4.864928e+06,0,1,-7689.513400,4.864929e+06,0,3,2.474478
5,-7691.33840,4.864928e+06,0,1,-7685.723500,4.864930e+06,0,0,3.729940
6,-7691.33840,4.864928e+06,0,1,-7684.298100,4.864932e+06,0,1,7.267768
7,-7691.33840,4.864928e+06,0,1,-7684.298100,4.864932e+06,0,2,9.227377
8,-7691.33840,4.864928e+06,0,1,-7684.298100,4.864932e+06,0,3,6.487093
9,-7691.33840,4.864928e+06,0,1,-7684.282400,4.864932e+06,0,0,2.107866


In [ ]:
data_all_x,data_all_y = df_WAP_pair.drop(['Y'], axis=1).values, df_WAP_pair['Y'].values
data_all_x.shape, data_all_y.shape

((869534, 8), (869534,))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data_all_x, data_all_y, test_size=0.1)
x_train.shape, x_test.shape

((782580, 8), (86954, 8))

# Encoder 

In [ ]:
num_inputs=4    #28x28 pixels
num_hid1=64
num_hid2=256
num_hid3=128
num_output=10
actf=tf.nn.relu

In [ ]:
X=tf.placeholder(tf.float32,shape=[None,num_inputs])
initializer=tf.variance_scaling_initializer()

w1=tf.Variable(initializer([num_inputs,num_hid1]),dtype=tf.float32)
w2=tf.Variable(initializer([num_hid1,num_hid2]),dtype=tf.float32)
w3=tf.Variable(initializer([num_hid2,num_hid3]),dtype=tf.float32)
w4=tf.Variable(initializer([num_hid3,num_output]),dtype=tf.float32)

b1=tf.Variable(tf.zeros(num_hid1))
b2=tf.Variable(tf.zeros(num_hid2))
b3=tf.Variable(tf.zeros(num_hid3))
b4=tf.Variable(tf.zeros(num_output))

hid_layer1=actf(tf.matmul(X,w1)+b1)
hid_layer2=actf(tf.matmul(hid_layer1,w2)+b2)
hid_layer3=actf(tf.matmul(hid_layer2,w3)+b3)
output_layer=actf(tf.matmul(hid_layer3,w4)+b4)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
X2=tf.placeholder(tf.float32,shape=[None,num_inputs])
initializer2=tf.variance_scaling_initializer()

w12=tf.Variable(initializer([num_inputs,num_hid1]),dtype=tf.float32)
w22=tf.Variable(initializer([num_hid1,num_hid2]),dtype=tf.float32)
w32=tf.Variable(initializer([num_hid2,num_hid3]),dtype=tf.float32)
w42=tf.Variable(initializer([num_hid3,num_output]),dtype=tf.float32)

b12=tf.Variable(tf.zeros(num_hid1))
b22=tf.Variable(tf.zeros(num_hid2))
b32=tf.Variable(tf.zeros(num_hid3))
b42=tf.Variable(tf.zeros(num_output))

hid_layer12=actf(tf.matmul(X2,w12)+b12)
hid_layer22=actf(tf.matmul(hid_layer12,w22)+b22)
hid_layer32=actf(tf.matmul(hid_layer22,w32)+b32)
output_layer2=actf(tf.matmul(hid_layer32,w42)+b42)

In [ ]:
dist = tf.sqrt(tf.reduce_sum(tf.square(output_layer2-output_layer), 1))
Y=tf.placeholder(tf.float32,shape=[None])
loss = tf.losses.mean_squared_error(Y,dist)

lr=0.00001
optimizer=tf.train.AdamOptimizer(lr)
train=optimizer.minimize(loss)

init=tf.global_variables_initializer()

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [ ]:
num_epoch=100
batch_size=512

with tf.Session() as sess:
    sess.run(init)
    n_sample = len(x_train)
    for epoch in range(num_epoch):
        order = shuffle_forward(len(x_train))
        x_train = x_train[order]
        y_train = y_train[order]
            
        num_batches = n_sample//batch_size
        for iteration in range(num_batches):
            start = iteration * batch_size
            end = min(n_sample, start + batch_size)
            X_batch,y_batch=x_train[start:end], y_train[start:end]
            sess.run(train,feed_dict={X:X_batch[:, :num_inputs], 
                                      X2:X_batch[:, num_inputs:],
                                      Y :y_batch})
        
        r_order = shuffle_backward(order)
        x_train = x_train[r_order]
        y_train = y_train[r_order]
            
        train_loss=loss.eval(feed_dict={X:x_test[:, :num_inputs], 
                                        X2:x_test[:, num_inputs:],
                                        Y :y_test})
        print("epoch {} loss {}".format(epoch,train_loss))

epoch 0 loss 3889.240966796875
epoch 1 loss 3743.5654296875
epoch 2 loss 3731.04638671875
epoch 3 loss 3729.112060546875
epoch 4 loss 3749.2138671875
epoch 5 loss 3735.214599609375
epoch 6 loss 4229.12255859375
epoch 7 loss 4612.578125
epoch 8 loss 820232.5625
epoch 9 loss 5598.41162109375
epoch 10 loss 4972.36767578125
epoch 11 loss 18483.4296875
epoch 12 loss 4313.4296875
epoch 13 loss 7283.93798828125
epoch 14 loss 11843.2978515625
